In [1]:
from whatever import *
from toolz.curried import *
from sklearn.base import ClassifierMixin, RegressorMixin
from sklearn.pipeline import make_union, Pipeline
from sklearn.ensemble import VotingClassifier
from pandas import np

__version__ = "0.0.1"

__all__ = []

In [24]:
class SimplePipeline(Pipeline):
    """Build a pipeline from a list."""
    def __init__(self, pipeline, n_jobs=4):
        pipeline = list(pipeline)
        
        if _X(pipeline[-1]) * (
            lambda x: isinstance(x, (
                    RegressorMixin, ClassifierMixin,
                ))
        ) | all > identity:            
            pipeline[-1] = VotingClassifier(
                _X(pipeline[-1]) * [str, identity] > list,
            )   
            
        pipeline = _X(pipeline) * callables.Dispatch([
            [VotingClassifier, identity],
        ], lambda m: make_union(*m).set_params(n_jobs=n_jobs)) > list
        
        self.pipeline = pipeline
        self.n_jobs = n_jobs
        super().__init__(
            _X(pipeline) * [str, identity] > list
        )
        

In [65]:
weiner_khinchin_auto_correlation = (
    _X(im) | np.fft.fftn | np.abs | (lambda x: x**2) | np.fft.ifftn | np.real
)[np.around](decimals=13)

In [151]:
windowed = weiner_khinchin_auto_correlation.copy()

def auto_correlation(im, s=(20,20,)):
    for i, token in enumerate(windowed._tokens):
        if first(token) in (np.fft.fftpack.fftn,):
            windowed._tokens[i][2] = {'s': s}
        if first(token) in (np.fft.fftpack.ifftn,):
            windowed._tokens[i][2] = {'s': s}
    return windowed.value(im)           

In [154]:
def cross_correlation(X, Y, s=(20, 20,)):    
    fft_based_cross_correlation = (
        _X(np.fft.fftn(X, s=s), np.fft.fftn(Y, s=s).conj())
         | np.multiply
         | np.fft.ifftn
         | np.real
        )[np.around](decimals=13)
    return fft_based_cross_correlation.value()

In [174]:
assert np.isclose(padded_autocorrelation(im), cross_correlation(im, im)).all()

In [152]:
windowed = weiner_khinchin_auto_correlation.copy()

def auto_correlation_np(im, s=(20,20,)):
    axes=_X(s).len.add(1).range(1).list.value()
    for i, token in enumerate(windowed._tokens):
        if first(token) in (np.fft.fftpack.fftn,):
            windowed._tokens[i][2] = {'s': s, 'axes':axes}
        if first(token) in (np.fft.fftpack.ifftn,):
            windowed._tokens[i][2] = {'s': s, 'axes': axes}
    return windowed.value(im)           

In [153]:
def cross_correlation_np(X, Y, s=(20, 20,)):    
    axes=_X(s).len.add(1).range(1).list.value()
    fft_based_cross_correlation_np = (
        _X(np.fft.fftn(X, s=s, axes=axes), np.fft.fftn(Y, s=s, axes=axes).conj())
         | np.multiply
         | np.fft.ifftn
         | np.real
        )[np.around](decimals=13)
    return fft_based_cross_correlation_np.value()

In [180]:
A = np.random.randn(30,10,10) > 1
B = auto_correlation_np(A)
assert np.isclose(
    np.apply_over_axes(np.sum, A, axes=[1,2]).ravel(),
    np.apply_over_axes(np.max, B, axes=[1,2]).ravel(),
).all()

In [121]:
correlation = callables.Dispatch({
        (np.ndarray,): padded_autocorrelation,
        (np.ndarray, np.ndarray): cross_correlation,
    })

In [137]:
%%time
_X((np.random.randn(20, 100, 100)>1).astype(int)).map(
    auto_correlation).list().value();

CPU times: user 10 ms, sys: 1.19 ms, total: 11.2 ms
Wall time: 10.1 ms


[array([[ 64.,  12.,   9.,   9.,  10.,  14.,   9.,  12.,  10.,  10.,   8.,
          10.,  10.,  12.,   9.,  14.,  10.,   9.,   9.,  12.],
        [ 12.,  14.,   9.,   9.,  10.,  12.,   9.,   8.,   9.,   9.,  11.,
          10.,   5.,   7.,   9.,   8.,   6.,   9.,  12.,  11.],
        [  9.,   9.,  12.,   6.,   9.,  12.,  10.,   8.,  10.,  10.,  11.,
          11.,  14.,  10.,  10.,   9.,  10.,  16.,  15.,  14.],
        [ 14.,   6.,  11.,   8.,   5.,  13.,   9.,  12.,   8.,   7.,  10.,
           7.,   7.,  12.,   8.,  16.,  11.,  10.,  13.,  11.],
        [ 11.,   9.,  11.,  13.,   8.,  11.,  10.,   9.,  13.,  10.,  11.,
           5.,  10.,   7.,  12.,  13.,  12.,  14.,   9.,   8.],
        [  7.,   9.,   9.,   4.,  13.,  14.,  10.,  10.,  14.,   6.,  10.,
           8.,  12.,   9.,  13.,  11.,  10.,  12.,   9.,  13.],
        [ 10.,   7.,  10.,  10.,   7.,  15.,   8.,   8.,  11.,   9.,  12.,
           9.,  12.,  11.,  10.,  12.,  10.,  10.,   9.,   6.],
        [  9.,  12.,  14., 

In [114]:
assert np.isclose(correlation(im), correlation(im, im)).all()

In [116]:
assert np.isclose(correlation(im, s=(31,31)), correlation(im, im, s=(31,31))).all()
assert correlation(im, s=(31,31)).shape==correlation(im, im, s=(31,31)).shape

In [26]:
pipeline = SimplePipeline([
        [decomposition.IncrementalPCA(), manifold.Isomap()],
    ])
assert isinstance(pipeline.steps[-1][1], base.TransformerMixin)

In [27]:
pipeline = SimplePipeline([
        [decomposition.IncrementalPCA(), manifold.Isomap()],
        [tree.DecisionTreeClassifier(), tree.DecisionTreeRegressor()],
    ])
assert isinstance(pipeline.steps[-1][1], base.ClassifierMixin)

In [32]:
pipeline = SimplePipeline([
        [decomposition.IncrementalPCA(), manifold.Isomap()],
        [tree.DecisionTreeClassifier(), tree.DecisionTreeRegressor()],
    ])
assert pipeline.set_params(n_jobs=2).n_jobs==2

In [33]:
%%time
pipeline.set_params(n_jobs=1).fit_transform(
    *_X(datasets.load_digits()).get(['data', 'target']).value()
)

CPU times: user 388 ms, sys: 353 ms, total: 741 ms
Wall time: 2.32 s


array([[ 0.,  0.],
       [ 1.,  1.],
       [ 2.,  2.],
       ..., 
       [ 8.,  8.],
       [ 9.,  9.],
       [ 8.,  8.]])

In [12]:
%%time
pipeline.set_params(n_jobs=2).fit_transform(
    *_X(datasets.load_digits()).get(['data', 'target']).value()
)

CPU times: user 445 ms, sys: 325 ms, total: 771 ms
Wall time: 2.34 s


array([[ 0.,  0.],
       [ 1.,  1.],
       [ 2.,  2.],
       ..., 
       [ 8.,  8.],
       [ 9.,  9.],
       [ 8.,  8.]])